In [1]:
import numpy as np
import nltk
from nltk.tokenize import word_tokenize,sent_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [27]:
fd = open('big.txt','r')
text = fd.read()
fd.close()

# Data preprocessing

removing special characters

In [28]:
text = text[:100000].lower()

for i in '!@#$~%^&*()_+-=[]{}\|;:",/<>?\n':
    text = text.replace(i,' ')
text = text.replace("'",' ')
text = text.replace("  ",' ')
text = text.replace("  ",' ')

word tokenization

In [29]:
words = word_tokenize(text)
words = [word for word in words if len(word)!=0]
vocab = list(set(words))

word_index = {word:i for i,word in enumerate(vocab)}
index_word = {word_index[word]:word for word in word_index}
print(index_word[340],word_index['let'])
sents = [word_tokenize(sent) for sent in sent_tokenize(text)]

let 340


# Model Training

In [67]:
features=[]
labels=[]
window_size = 2
for sent in sents:
  for i in range(len(sent)-window_size*2):
     features.append(sent[i:i+window_size]+sent[i+window_size+1:i+window_size*2+1])
     labels.append(sent[i+window_size])

X_train = []
for feature in features:
    enc = np.zeros(len(word_index))
    for word in feature:
        enc[word_index[word]] = 1
    X_train.append(enc)

Y_train=[]
for word in labels:
  enc = np.zeros(len(word_index))
  enc[word_index[word]]=1
  Y_train.append(enc)

X_train = np.array(X_train)
Y_train = np.array(Y_train)


# Building the Neural Network

In [53]:
from keras.layers import Dense
from keras.models import Sequential

model = Sequential()
### Middle Layer
model.add(Dense(100,input_dim=len(word_index),activation='relu'))
### Output Layer
model.add(Dense(len(word_index),activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

model.fit(X_train,Y_train,epochs=10,batch_size=32)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
482/482 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.0528 - loss: 7.1255
Epoch 2/10
482/482 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.0713 - loss: 5.9820
Epoch 3/10
482/482 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0964 - loss: 5.6932
Epoch 4/10
482/482 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.1150 - loss: 5.4610
Epoch 5/10
482/482 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.1477 - loss: 5.1832
Epoch 6/10
482/482 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.1780 - loss: 4.8554
Epoch 7/10
482/482 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.2260 - loss: 4.5232
Epoch 8/10
482/482 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.2797 - loss: 4.0899
Epoch 9/10
482/482 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.3200 - loss: 3.7153
Epoch 10/10
482/482 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.3695 - loss: 3.2955


finding similar words

In [66]:
word_embeddings = model.get_weights()[0]
target_word = word_embeddings[word_index['him']]
distances= np.dot(word_embeddings,target_word)
most_similar_indices = np.argsort(distances)[::-1][:5]
print([index_word[i] for i in most_similar_indices])

['him', 'his', 'my', 'morrow', 'uncertain']
